In [ ]:

#malw or female
#change male
from google.colab import drive
import os
import cv2
import numpy as np
import pandas as pd
import os
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.losses import categorical_crossentropy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input
from keras.layers import *
from keras.models import *
from keras.models import load_model
from tensorflow.keras import layers, Model, callbacks
from keras import backend as K
import tensorflow as tf
import copy
from keras.layers import Input
from keras.layers.merge import concatenate

def number_of_class_train(trainlist):
  listofclass=[]
  cunt=[0,0,0,0,0,0,0,0,0,0,0]
  for i in range(len(trainlist)):
    if trainlist[i]==0:
      cunt[0]=cunt[0]+1
    if trainlist[i]==1:
      cunt[1]=cunt[1]+1
    if trainlist[i]==2:
      cunt[2]=cunt[2]+1
    if trainlist[i]==3:
      cunt[3]=cunt[3]+1
    if trainlist[i]==4:
      cunt[4]=cunt[4]+1
    if trainlist[i]==5:
      cunt[5]=cunt[5]+1
    if trainlist[i]==6:
      cunt[6]=cunt[6]+1
    if trainlist[i]==7:
      cunt[7]=cunt[7]+1
    if trainlist[i]==8:
      cunt[8]=cunt[8]+1
    if trainlist[i]==9:
      cunt[9]=cunt[9]+1
    if trainlist[i]==10:
      cunt[10]=cunt[10]+1

  for j in range(11):
    listofclass.append(cunt[j])

  return listofclass

def recall(permatrix,actualmat,num):
  dim=permatrix.shape
  tp,fp,fn=0,0,0
  for i in range(dim[0]):
    for j in range(dim[1]):
      if permatrix[i][num]==1 and actualmat[i][num]==1:
        tp=tp+1
      if actualmat[i][num]==0 and permatrix[i][num]==1:
        fp=fp+1
      if actualmat[i][num]==1 and permatrix[i][num]==0:
        fn=fn+1
  recall_num=tp/(tp+fn+(pow(10,-10)))
  return recall_num


def precision(permatrix,actualmat,num):
  dim=permatrix.shape
  tp,fp,fn=0,0,0
  for i in range(dim[0]):
    for j in range(dim[1]):
      if permatrix[i][num]==1 and actualmat[i][num]==1:
        tp=tp+1
      if actualmat[i][num]==0 and permatrix[i][num]==1:
        fp=fp+1
      if actualmat[i][num]==1 and permatrix[i][num]==0:
        fn=fn+1
  precision_num=tp/(tp+fp+(pow(10,-10)))
  return precision_num


def creat_modle(inputs):
  output=[]
  c1 = Conv2D(96, kernel_size=(5, 5),strides=3,activation='relu')(inputs)
  s2 = MaxPooling2D(pool_size=(3, 3),strides=(1,1))(c1)
  norm2 = tf.nn.lrn(s2,depth_radius=5, bias=1.0, alpha=0.0001 , beta=0.75,name='norm2')
  c3 = Conv2D(256, kernel_size=(5, 5),strides=3,activation='relu')(norm2)
  s4 = MaxPooling2D(pool_size=(3, 3),strides=(1,1))(c3)
  norm4 = tf.nn.lrn(s4,depth_radius=5, bias=1.0, alpha=0.0001 , beta=0.75,name='norm4')
  c5 = Conv2D(384, kernel_size=(3, 3),strides=3,activation='relu')(norm4)
  s6 = MaxPooling2D(pool_size=(3, 3),strides=(1,1))(c3)
  x = Dropout(0.25)(s6)
  flat = Flatten()(x)
  encoder_output=flat
  dropout = Dropout(0.5)
  age_model = Dense(512, activation='relu')(flat)
  age_model = dropout(age_model)
  age_model = Dense(512, activation='relu')(age_model)
  age_model = dropout(age_model)
  age_model = Dense(2, activation='softmax')(age_model)
  output.append(age_model)
  output.append(encoder_output)
  return  output


def fine_tuning(flat0,flat1,flat2,flat3):
  merge = concatenate([flat0, flat1,flat2,flat3])
  dropout = Dropout(0.5)
  hybrid_model = Dense(512, activation='relu')(merge)
  hybrid_model = dropout(hybrid_model)
  hybrid_model = Dense(512, activation='relu')(hybrid_model)
  hybrid_model = dropout(hybrid_model)
  output= Dense(5, activation='custom_activation')(hybrid_model)
  return output



drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/min"
path = "/content/gdrive/My Drive/min"
files = os.listdir(path)
size = len(files)

ages = []
genders = []
images =[]
def displayImage(image):
    plt.imshow(image)
    plt.show()

for file in files:

    image = cv2.imread( path+'/'+file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,dsize=(64,64))
    image = image.reshape((image.shape[0],image.shape[1],3))
    images.append(image)
    split_var = file.split('_')
    ages.append(split_var[0])
    genders.append(int(split_var[1]) )

from collections import Counter


counter_ages = dict(Counter(ages))

print("ages",ages)
print("counter",counter_ages)

listOfKeys = [key  for (key, value) in counter_ages.items() if value == 1]
print(listOfKeys)
print("10",counter_ages.get('10'))
print("12",counter_ages.get('12'))
print("99",counter_ages.get('99'))
print("101",counter_ages.get('101'))
print("110",counter_ages.get('110'))
print("112",counter_ages.get('112'))
print("76",counter_ages.get('76'))
print("78",counter_ages.get('78'))
print("2",counter_ages.get('2'))
print("4",counter_ages.get('4'))

idx = 44
sample = images[idx]
print("Age:",ages[idx])
def age_group(age):
    if age >=0 and age < 20:
        return 0
    elif 20<=age<30:
      return 1
    elif 30<=age < 40:
        return 2
    elif 40<=age < 50:
        return 3
    else:
      return 4


from keras.utils.generic_utils import get_custom_objects
def net_input(X, W, b):
    return X @ W + b


list_age=[]
for member in ages:
  if member==11:
    list_age.append(10)
  elif member==100:
    list_age.append(99)
  elif member==111:
    list_age.append(110)
  elif member==77:
    list_age.append(76)
  elif member==3:
    list_age.append(4)
  else:
    list_age.append(member)


def custom_activation(x):
  return tf.math.log(1.0 + (tf.math.exp(x) / (1.0 + tf.math.exp(x))))

print("ages",ages)
print("list_age",list_age)
counter_ages1 = dict(Counter(list_age))
print("counter1",counter_ages1)
listOfKeys1 = [key  for (key, value) in counter_ages1.items() if value == 1]
print("listOfKeys1",listOfKeys1)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})


target = []
features = np.zeros((size,sample.shape[0],sample.shape[1],3),dtype = 'float32')
for i in range(size):
#    target[i,0] = age_group(int(ages[i])) / 10
    target.append(int(ages[i]))

    features[i] = images[i]



print("lisage",target)
features = features / 255
display(sample)


print("ages",ages)
print("target",target)

#x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.1,random_state=42,shuffle  = True,stratify=target)


def nll1(y_test, peredict_target):
    return K.sum(K.binary_crossentropy(y_test, peredict_target), axis=-1)


x_train, x_test, y_train, y_test = train_test_split(features,target, test_size=0.1,shuffle  = True,stratify=target)
num_folds=5
from sklearn.model_selection import KFold
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate(( y_train, y_test), axis=0)
kfold = KFold(n_splits=num_folds, shuffle=False)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
  models=[]
#  inputs = Input(shape=(64,64,3))
  encoder_input = keras.Input(shape=(64, 64, 3), name="img")
  print("ytrain",y_train)
  print("ytest",y_test)
  from keras.utils import np_utils
  y_train0=[]
  y_train1=[]
  y_train2=[]
  y_train3=[]
  lable_age=[]
  weights0=[]
  weights1=[]
  weights2=[]
  weights3=[]
  weights=[]
  encoder=[]
  trainrcnn=[]
  for member in y_test:
    lable_age.append(age_group(member))
  testY=tf.keras.utils.to_categorical(lable_age, num_classes=5, dtype='float32')
  for member in y_train:
    trainrcnn.append(age_group(member))
  trainY_org=tf.keras.utils.to_categorical(trainrcnn, num_classes=5, dtype='float32')
  for i in range(4):
    if i==0 :
      y_train0=copy.deepcopy( y_train)
      for j in range(len(y_train)):
        if y_train[j]<=19 :
          y_train0[j]=0
        else :
          y_train0[j]=1
      trainY=tf.keras.utils.to_categorical(y_train0, num_classes=2, dtype='float32')
      #s0
      input = creat_modle(encoder_input)
      age_model_0 = input[0]
      encoder_output_0 = input[1]
      model0 = Model(inputs=encoder_input, outputs=[age_model_0])
      model0.compile(optimizer ='SGD', loss =nll1, metrics='acc')
      model0.fit(x_train,trainY,batch_size=28,epochs=30)
      models.append(model0)
      for layer in model0.layers:
        weights0.append(layer.get_weights())
        weights.append(weights0)
        encoder.append(encoder_output_0)
    if i==1 :
      y_train1=copy.deepcopy( y_train)
      for j in range(len(y_train)):
        if y_train[j]<=29 :
          y_train1[j]=0
        else :
          y_train1[j]=1
      trainY=tf.keras.utils.to_categorical(y_train1, num_classes=2, dtype='float32')
      #s1
      input = creat_modle(encoder_input)
      age_model_1 = input[0]
      encoder_output_1 = input[1]
      model1 = Model(inputs=encoder_input, outputs=[age_model_1])
      model1.compile(optimizer ='SGD', loss =nll1, metrics='acc')
      model1.fit(x_train,trainY,batch_size=28,epochs=30)
      models.append(model1)
      for layer in model1.layers:
        weights1.append(layer.get_weights())
        weights.append(weights1)
        encoder.append(encoder_output_1)
    if i==2 :
      y_train2=copy.deepcopy( y_train)
      for j in range(len(y_train)):
        if y_train[j]<=39 :
          y_train2[j]=0
        else :
          y_train2[j]=1
      trainY=tf.keras.utils.to_categorical(y_train2, num_classes=2, dtype='float32')
      #s2
      input = creat_modle(encoder_input)
      age_model_2 = input[0]
      encoder_output_2 = input[1]
      model2 = Model(inputs=encoder_input, outputs=[age_model_2])
      model2.compile(optimizer ='SGD', loss =nll1, metrics='acc')
      model2.fit(x_train,trainY,batch_size=28,epochs=30)
      models.append(model2)
      for layer in model2.layers:
        weights2.append(layer.get_weights())
        weights.append(weights2)
        encoder.append(encoder_output_2)
    if i==3 :
      y_train3=copy.deepcopy( y_train)
      for j in range(len(y_train)):
        if y_train[j]<=49 :
          y_train3[j]=0
        else :
          y_train3[j]=1
      trainY=tf.keras.utils.to_categorical(y_train3, num_classes=2, dtype='float32')
      #s3
      input = creat_modle(encoder_input)
      age_model_3 = input[0]
      encoder_output_3 = input[1]
      model3 = Model(inputs=encoder_input, outputs=[age_model_3])
      model3.compile(optimizer ='SGD', loss =nll1, metrics='acc')
      model3.fit(x_train,trainY,batch_size=28,epochs=30)
      models.append(model3)
      for layer in model3.layers:
        weights3.append(layer.get_weights())
        weights.append(weights3)
        encoder.append(encoder_output_3)

      #s4
      rankingcnn=fine_tuning(encoder[0],encoder[1],encoder[2],encoder[3])
      model4 = Model(inputs=encoder_input, outputs=[rankingcnn])
      model4.compile(optimizer ='SGD', loss =nll1, metrics='acc')
      model4.fit(x_train,trainY_org,batch_size=28,epochs=30)
      models.append(rankingcnn)
  print("x_train.shape",x_train.shape)
  print("x_test.shape",x_test.shape)

  fold_no = fold_no + 1






#end
predict_targets=[]
rx=[]
for i in range(len(models)-1):
  predict_targets.append(models[i].predict(x_test))
for i in range(len(y_test)):
  sum=0
  for j in range(len(predict_targets)):
    sum+=predict_targets[j][i]

  rx.append(1+sum)


peredict_model4=model4.predict(x_test)
print("per",rx)
print("peredict_model4",peredict_model4)
print("y",testY)
print('\n')
#model.evaluate(x_test, testY)
#print(model.evaluate)
print("counter y_train0 (<=19, >19)",dict(Counter(y_train0)))
print("counter y_train1 ( <=29 , >29)",dict(Counter(y_train1)))
print("counter y_train2 (<=39, >39 )",dict(Counter(y_train2)))
print("counter y_train3 (<=49 , >49 )",dict(Counter(y_train3)))











#this is